In [1]:
import json

In [9]:
with open("HP_KG_7_books.json", "r", encoding="utf-8") as f:
    data = json.load(f)


# Step 2: Convert to a dictionary with numbered integer keys
def transform_to_numbered_dict(data):
    return {i: item for i, item in enumerate(data, start=1)}

numbered_data = transform_to_numbered_dict(data)

# Step 3: Write the transformed data to a new JSON file
with open("output.json", "w", encoding="utf-8") as f:
    json.dump(numbered_data, f, indent=4)